In [1]:
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy 
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import pandas as pd 

In [2]:
r_df = pd.read_csv('data/ratings.csv')
m_df = pd.read_csv('data/movies.csv')
t_df = pd.read_csv('data/tags.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/ratings.csv'

In [14]:
df = pd.merge(r_df, m_df[['movieId', 'genres']], on = 'movieId', how = 'left')

df

,userId,movieId,rating,timestamp,genres
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Comedy|Romance
2,1,6,4.0,964982224,Action|Crime|Thriller
3,1,47,5.0,964983815,Mystery|Thriller
4,1,50,5.0,964982931,Crime|Mystery|Thriller
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Drama|Horror|Thriller
100832,610,168248,5.0,1493850091,Action|Crime|Thriller
100833,610,168250,5.0,1494273047,Horror
100834,610,168252,5.0,1493846352,Action|Sci-Fi


In [9]:
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
mlb = MultiLabelBinarizer()

df['userId'] = user_encoder.fit_transform(df['userId'])
df['movieId'] = movie_encoder.fit_transform(df['movieId'])

df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('genres').str.split('|')), columns = mlb.classes_, index = df.index ))

In [13]:
df

,userId,movieId,rating,timestamp,Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,4.0,964982703,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,4.0,964981247,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,0,5,4.0,964982224,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,0,43,5.0,964983815,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,0,46,5.0,964982931,0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,609,9416,4.0,1493848402,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
100832,609,9443,5.0,1493850091,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
100833,609,9444,5.0,1494273047,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
100834,609,9445,5.0,1493846352,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [15]:
train_df, test_df = train_test_split(df, test_size = 0.2)
train_df

,userId,movieId,rating,timestamp,genres
20660,136,239,1.0,832450019,Animation|Children|Comedy|Romance
25931,179,500,4.0,840907726,Comedy|Drama
67132,434,1036,4.5,1270606799,Action|Crime|Thriller
70118,448,101415,1.5,1377355598,Comedy|Drama|Romance
34115,232,4896,3.5,1218166089,Adventure|Children|Fantasy
...,...,...,...,...,...
61488,405,6807,4.0,1300896094,Comedy
94124,599,6305,2.5,1519135714,Drama|Sci-Fi
81946,518,2657,4.5,1056907463,Comedy|Horror|Musical|Sci-Fi
50496,326,2338,2.0,1322250784,Horror|Mystery|Thriller


In [16]:
reader = Reader(rating_scale = (0.5, 5))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()

In [17]:
model_svd = SVD()
model_svd.fit(trainset)

predictions_svd = model_svd.test(trainset.build_anti_testset())
accuracy.rmse(predictions_svd)

RMSE: 0.4742


0.47415498426144354

In [18]:
def get_recs(user_id, n=10):
  user_movies = df[df['userId'] == user_id]['movieId'].unique()
  all_movies = df['movieId'].unique()
  movies_predict = list(set(all_movies) - set(user_movies))

  user_movie_pairs = [(user_id, movie_id, 0) for movie_id in movies_predict]
  predictions_cf = model_svd.test(user_movie_pairs)

  top_n_recommendations = sorted(predictions_cf, key = lambda x: x.est)[:n]

  for pred in top_n_recommendations:
    predicted_rating = pred.est
    print(predicted_rating)

  top_n_movie_ids = [int(pred.iid) for pred in top_n_recommendations]

  top_n_movies = movie_encoder.inverse_transform(top_n_movie_ids)

  return top_n_movies 


In [21]:
user_id = 221
recommendations = get_recs(user_id)
top_n_movies_titles = m_df[m_df['movieId'].isin(recommendations)]['title'].tolist()
print(f"Top 10 Recommendations for User {user_id}:")
for i, title in enumerate(top_n_movies_titles, 1):
  print(f"{i}.{title}")
     

2.2780387558253326
2.555733056995654
2.5559078119551897
2.599539605472597
2.700440704234268
2.7118798729294147
2.712969692618455
2.726902009897745
2.7306898165263616
2.7621974200817476
Top 10 Recommendations for User 221:
1.Coneheads (1993)
2.Kazaam (1996)
3.Grease 2 (1982)
4.Anaconda (1997)
5.Speed 2: Cruise Control (1997)
6.Batman & Robin (1997)
7.Godzilla (1998)
8.Superman IV: The Quest for Peace (1987)
9.Problem Child 2 (1991)
10.Battlefield Earth (2000)
